Алексей Сек, БЭК182

## Preparation: importing packages and dataset

In [268]:
# Importing packages
import pandas as pd
import scipy as sp
import scipy.linalg as spla
import scipy.optimize as spopt
import scipy.stats as spst
import numpy as np
import numpy.linalg as npla
from sklearn.linear_model import LinearRegression as LR
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import Ridge as Ridge
from sklearn.model_selection import KFold
import itertools

In [269]:
# Get sample dataset
import statsmodels.api as sm
dat = sm.datasets.get_rdataset("Guerry", "HistData").data

In [270]:
smfOLS = sm.regression.linear_model.OLS.from_formula

In [271]:
dat.shape

(86, 23)

# Important: I left comments in the code + Added markdowns starting with "Steps:" for my notes about the solution

Also I copied useful markdowns from the seminar, hopefully, this won't be trated as plagiat :)

# Assignment 1 (2 points).
Write a function assignment_1 that replicates the outcome of assignment_1_true on the *dat* dataset from the beginning of the notebook

In [6]:
def assignment_1_true(formula, data):
    
    fit = smfOLS(formula, data = data).fit()
    
    return fit.conf_int()

In [7]:
assignment_1_true('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat)

,0,1
Intercept,-6.027115,41.046645
Literacy,-0.479342,0.179528
Donations,-0.000675,0.000954
Infants,-0.000005,0.001164
Wealth,0.096173,0.516922
Commerce,-0.100867,0.405366


### Let's leverage formulas from the seminar to derive Confidence Intervals

- prediction for unlabeled data Z is merely $$ Z \beta = Z (X'X)^{-1} (X'X)$$
- confidence intervals are $$\beta_0 = \beta \pm 1.96*\sqrt{diag \ Cov(\beta)}, \quad \hat{Cov}(\beta) = \sigma^2 (X'X)^{-1}, \quad \hat \sigma = \frac{e'e}{n-p}$$

### Steps:
1. Estimate beta_hat with closed form formula for OLS regression
2. Find variances of beta (they are located on the diagonal of covariance matrix)
    - Firstly get e (Y - X$\beta$)
    - Caclulate variance (square of st.dev): X.shape[0] used to get size of X matrix
    - Define covariance matrix and then get diagonal from it
3. 1.96 is an estimation of t-crit when number of observations $\rightarrow \infty$
    - However, we have only 86 observations, so better to define exact t_crit, so use scipy.stats package
4. Calculate Confidence intervals using simple formula mentioned above

In [57]:
def assignment_1(formula, data):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    # my code
    # we need to derive left and right ends of Confidence intervals     
    
    # actually we need firstly to find the beta_hat matrix and then calculate CIs based on it
    beta_hat = spla.inv(X.T@X)@(X.T@Y)
    
    # this is derived from the formula above, also used the following article: https://web.stanford.edu/~mrosenfe/soc_meth_proj3/matrix_OLS_NYU_notes.pdf
    e = Y - X@beta_hat
    sigma_sq = (e@e.T)/(X.shape[0] - X.shape[1])
    cov_beta = sigma_sq * spla.inv(X.T@X)
    
    # find the intervals
    t_stat = abs(sp.stats.t.ppf(0.025,80))
    left = beta_hat - t_stat * np.sqrt(np.diagonal(cov_beta))
    right = beta_hat + t_stat * np.sqrt(np.diagonal(cov_beta))
    
    # your mock results
    results = [left, right]
    
    df = pd.DataFrame(results)
    # changed code a bit because it was more convenient for me to take results (2 x 6) and then transponse to (6 x 2)    
    df = df.T
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [58]:
assignment_1('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat)

,0,1
Intercept,-6.027115,41.046645
Literacy,-0.479342,0.179528
Donations,-0.000675,0.000954
Infants,-0.000005,0.001164
Wealth,0.096173,0.516922
Commerce,-0.100867,0.405366


# Assignment 2 (3 points).
- Write a function that finds the coefficients for the **elastic net** regularized ols with coefficients $\lambda$ and $\mu$ of your choice
- This time you do not need to find the standard errors

### Elastic-net - formulas from the seminar
$$L = ||Y - X \beta ||^2 + \lambda ||\beta||^2 + \mu |\beta|_0\to \min$$
- increasing $\lambda$ fights multicollinearity but tends to spread the coefficients evenly
- increasing $\mu$ tends to find corner solutions (on kinks) thus prioritizes one coefficient over another

### Steps
1. Define loss function
    - For me the struggle was how to add $\mu|\beta|$. To get $|\beta|$ I used numpy package (linalg.norm function)
2. Choose any lamda and mu hyperparameters: I rook 3 and 5 correspondingly
3. Use any global minimization function - I took example from the seminar with spopt.shgo and modified it by adding mu and lambda
4. Results is actually a vector $\beta$ (vector of coefficients)

In [250]:
# took function from the seminar for OLS and added Regularization to it
def loss(Y, X, beta, lamda, mu):
    z = Y - X@beta
    return z@z.T + lamda*beta@beta.T + mu*np.linalg.norm(beta)
# + mu*beta

In [258]:
def assignment_2(formula, data, lamda, mu):
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    # my code
    # actually let's use the same code as in the seminar but add lamda and mu to it
    covs = 6
    bounds = [(None, None)]*covs
    results = spopt.shgo(lambda b: loss(Y, X, b, lamda, mu), bounds).x
    
    
    # your mock results
    df = pd.DataFrame(results, columns = [0])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [259]:
assignment_2('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat, 3, 5)

,0
Intercept,8.499647
Literacy,-0.042766
Donations,0.000161
Infants,0.000684
Wealth,0.315037
Commerce,0.197951


# Assignment 3 (5 points)
- Write a function that finds the coefficients for the **elastic net** regularized ols with crossvalidation
- Use number of folds of your choice
- You do not need to find the standard errors
- Search for $\lambda$, $\mu$ in the range [0,5]x[0,5]

### Steps:
1. Define loss function same as for assignment 2
2. Define a function that optimizes betas for one fold (later we will use them in a loop for k folds)
    - After getting optimized betas we can get a metric that we want to optimize (I used SSR as in the seminar)
3. Iterate over different combinations of mu and lambda and save the values of metric that we are minimizing (SSR)
4. Find what mu and lamda correspond to the lowest metric value. These are optimal hyperparameters found with CV
    - Actually it is optimization (minimization) of error by mu and lamda
5. Take the entire dataset and run optimization of betas for elastic net with optimal mu and lamda
    - Here we use global optimization again as far as elnet has not closed form solution
    - Results is the vector of betas

In [272]:
# loss function for elastic net

def loss(Y, X, beta, lamda, mu):
    z = Y - X@beta
    return z@z.T + lamda*beta@beta.T + mu*np.linalg.norm(beta)

In [278]:
# Actually this is partially the code from the seminar but adjusted to elastic net
# As far for elastic net there is no closed form solution - I used global optimization to get betas

def run(lamda, mu, X, Y, kf):
    
    avg = 0
    
    for train_index, test_index in kf.split(X):
        
        X_train, X_test = X[train_index], X[test_index]
        Y_train, Y_test = Y[train_index], Y[test_index]
        
        # initially betas were for Ridge, now I adjusted to elastic net
        # important that we use X_train for beta coefficient there
        covs = 6
        bounds = [(None, None)]*covs
        beta = spopt.shgo(lambda b: loss(Y_train, X_train, b, lamda, mu), bounds).x
        
        # important that residuals with test observations, not train ones        
        errors = X_test@beta - Y_test
        SSR = errors@errors.T
        avg = avg + SSR
        
    # get average from all our folds
    return avg/kf.n_splits

In [276]:
def assignment_3(formula, data, folds = 5):
    
    # this code was there initially    
    ycolumn = formula.split(' ~ ')[0]
    xcolumns = formula.split(' ~ ')[1].split(' + ')
    Y = data[ycolumn].values
    X = data[xcolumns].values
    X = sm.add_constant(X)
    
    # my code
    lamdas = []
    mus = []
    results = []
    
    # fold our dataset     
    kf = KFold(n_splits=folds)
    
    # iterate over different lamda and mu combinations    
    for lamda, mu in itertools.product(np.linspace(0,5,20), np.linspace(0,5,20)):
        results.append(run(lamda, mu, X, Y, kf))
        lamdas.append(lamda)
        mus.append(mu)
    
    # finally get optimal lamda and mu (to define it we need to check for what lamda and mu corresponds to the lowest SSR)    
    opt_lamda = np.round(lamdas[np.argmin(results)],2)
    opt_mu = np.round(mus[np.argmin(results)],2)
    
    
    # using optimal hyperparameters we can find coefficients of the elastic net
    # actually this is the code from assignment 2 but here we use optimal mu and lamda, not randomly chosen ones
    covs = 6
    bounds = [(None, None)]*covs
    coefficients = spopt.shgo(lambda b: loss(Y, X, b, opt_lamda, opt_mu), bounds).x
    
    # this code was there initially
    # your mock results
    results = coefficients
    
    df = pd.DataFrame(results, columns = [0])
    df.index = np.array(['Intercept'] + xcolumns)
    
    return df

In [277]:
# elastic net coefficients with optimal mu and lamda
assignment_3('Lottery ~ Literacy + Donations + Infants + Wealth + Commerce', dat, folds = 3)

KeyboardInterrupt: 

# Assignment 4 (5 points).
- Write a function that finds the coefficients for the **ridge** regularized ols with coefficient $\lambda$ crossvalidated
- This time you HAVE TO to find the standard errors
- Derive the standard errors in markdown, assuming homoskedasticity of errors $E[ee'|X]=\sigma^2 I$
- see, for example, https://lukesonnet.com/teaching/inference/200d_standard_errors.pdf

# I could not manage with completing this task